# Heart Failiure prediciton model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing dependencies

In [ ]:
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
import numpy as np
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV

### Loading the dataset

In [ ]:
df=pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
x = df.drop("DEATH_EVENT", axis = 1)
y = df.DEATH_EVENT

### Scaling features (making them all of equal scale)

In [ ]:
scaler = MinMaxScaler()
x[['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']] = scaler.fit_transform(x[['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']])

### Mean encoding

In [ ]:
def mean_encoding(dataset, collumnname_for_encoding, label_collumn_name):
    mean_encoded_collum = df.groupby([collumnname_for_encoding])[label_collumn_name].mean().to_dict()
    return mean_encoded_collum

In [ ]:
anaemia_mean_encoding = mean_encoding(df, "anaemia", "DEATH_EVENT")
anaemia_mean_encoding = x["anaemia"].map(anaemia_mean_encoding)

diabetes_mean_encoding = mean_encoding(df, "diabetes", "DEATH_EVENT")
diabetes_mean_encoding = x["diabetes"].map(diabetes_mean_encoding)

high_blood_pressure_mean_encoding = mean_encoding(df, "high_blood_pressure", "DEATH_EVENT")
high_blood_pressure_mean_encoding = x["high_blood_pressure"].map(high_blood_pressure_mean_encoding)

sex_mean_encoding = mean_encoding(df, "sex", "DEATH_EVENT")
sex_mean_encoding = x["sex"].map(sex_mean_encoding)

smoking_mean_encoding = mean_encoding(df, "smoking", "DEATH_EVENT")
smoking_mean_encoding = x["smoking"].map(smoking_mean_encoding)

In [ ]:
x['anaemia_mean_encoding'] = anaemia_mean_encoding
x['diabetes_mean_encoding'] = diabetes_mean_encoding
x['high_blood_pressure_mean_encoding'] = high_blood_pressure_mean_encoding
x['sex_mean_encoding'] = sex_mean_encoding
x['smoking_mean_encoding'] = smoking_mean_encoding

### Removal of outliers via the Z score

In [ ]:
z_score = np.abs(stats.zscore(x))
location_of_outliers = np.where(z_score > 3)
x.drop(location_of_outliers[0], inplace = True)
y.drop(location_of_outliers[0], inplace = True)

In [ ]:
new_x, new_y = SMOTENC(categorical_features=[1,3,5,9,10]).fit_resample(x, y)

### Splitting the data into training and test sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, test_size = 0.2, random_state = 25)

### Modeling

In [ ]:
random_forest = RandomForestClassifier(max_depth=12, random_state=25)
random_forest.fit(x_train, y_train)
y_hat = random_forest.predict(x_test)
Accuracy = accuracy_score(y_test, y_hat)
print("Accuracy : ",Accuracy)
print(accuracy_score(y_train, random_forest.predict(x_train)))

Was going to use residual boosting but due to the train accuracy being so high (100%) there is no point, due to the second algorithm having no errors to work with